In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

/Users/user/Documents/Project/Environments/env_GenAI_Chatbot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [134]:

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
os.environ["NOMIC_API_KEY"] = os.getenv("NOMIC_API_KEY")

In [120]:
def get_text_chunks(text,ids,file_name):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=50000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    ids_current = [f"{file_name}.{l}" for l in range(len(chunks))]
    ids+=ids_current
    return chunks,ids


In [144]:
def get_pdf_text(pdf_docs,folder_path):
    ids=[]
    chunks_all=[]
    for pdf in pdf_docs:
        pdf_reader= PdfReader(os.path.join(folder_path,pdf))
        print(f"Processing file - {pdf}")
        text=""
        for page in pdf_reader.pages:
            text+= page.extract_text()
        chunks,ids = get_text_chunks(text,ids,pdf)
        chunks_all+=chunks
    return  text,chunks_all,ids


In [135]:
#---- Googel/Nomic API key -----
from langchain_nomic import NomicEmbeddings

embeddings_google = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
embeddings_nomic = NomicEmbeddings(model="nomic-embed-text-v1.5")

In [151]:
def get_vector_store(embeddings_api,text_chunks,ids):
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings_api,ids=ids)
    #vector_store = FAISS.from_documents(documents=text_chunks , embedding=embeddings_api)
    
    #vector_store.save_local("faiss_index")
    return vector_store


In [137]:
def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain


In [123]:
folder_path = "/Users/user/Documents/Project/Multi_PDFs_ChatApp_AI_Agent/docs"
files_in_folder = os.listdir("docs")
data,chunks,ids = get_pdf_text(files_in_folder,folder_path)
data


Processing file - AGI.pdf
Processing file - Understanding LLMs.pdf
Processing file - ComputerVision.pdf
Processing file - AI Assistants.pdf
Processing file - Augmented LLMs.pdf


'LLM A UGMENTED LLM S:\nEXPANDING CAPABILITIES THROUGH COMPOSITION\nRachit Bansal1Bidisha Samanta1Siddharth Dalmia2Nitish Gupta1Shikhar Vashishth1\nSriram Ganapathy1Abhishek Bapna1Prateek Jain1Partha Talukdar1\n1Google Research2Google DeepMind\nABSTRACT\nFoundational models with billions of parameters which have been trained on large\ncorpora of data have demonstrated non-trivial skills in a variety of domains. How-\never, due to their monolithic structure, it is challenging and expensive to augment\nthem or impart new skills. On the other hand, due to their adaptation abilities,\nseveral new instances of these models are being trained towards new domains and\ntasks. In this work, we study the problem of efficient and practical composition\nof existing foundation models with more specific models to enable newer capa-\nbilities. To this end, we propose CALM—Composition to Augment Language\nModels—which introduces cross-attention between models to compose their rep-\nresentations and ena

In [147]:
len(chunks)
ids

['AGI.pdf.0',
 'AGI.pdf.1',
 'AGI.pdf.2',
 'AGI.pdf.3',
 'AGI.pdf.4',
 'Understanding LLMs.pdf.0',
 'Understanding LLMs.pdf.1',
 'Understanding LLMs.pdf.2',
 'ComputerVision.pdf.0',
 'ComputerVision.pdf.1',
 'AI Assistants.pdf.0',
 'AI Assistants.pdf.1',
 'AI Assistants.pdf.2',
 'AI Assistants.pdf.3',
 'Augmented LLMs.pdf.0',
 'Augmented LLMs.pdf.1']

In [152]:
vector_store = get_vector_store(embeddings_nomic,chunks,ids)

In [129]:
print(f"Embedding function - {vector_store.embedding_function}")
print(f"Index of doc - {vector_store.index_to_docstore_id}")

Embedding function - <langchain_nomic.embeddings.NomicEmbeddings object at 0x127751fa0>
Index of doc - {0: 'AGI.pdf.0', 1: 'AGI.pdf.1', 2: 'AGI.pdf.2', 3: 'AGI.pdf.3', 4: 'AGI.pdf.4', 5: 'Understanding LLMs.pdf.0', 6: 'Understanding LLMs.pdf.1', 7: 'Understanding LLMs.pdf.2', 8: 'ComputerVision.pdf.0', 9: 'ComputerVision.pdf.1', 10: 'AI Assistants.pdf.0', 11: 'AI Assistants.pdf.1', 12: 'AI Assistants.pdf.2', 13: 'AI Assistants.pdf.3', 14: 'Augmented LLMs.pdf.0', 15: 'Augmented LLMs.pdf.1'}


In [130]:
#user_question = input("Ask a Question from the PDF Files uploaded - ")
user_question = "what is Sign Language Understanding in computer vision?"
print(user_question)

what is Sign Language Understanding in computer vision?


In [131]:
docs = vector_store.similarity_search(user_question)

chain = get_conversational_chain()
response = chain(
    {"input_documents":docs, "question": user_question}
    , return_only_outputs=True)

print(response)


{'output_text': 'Based on the provided text, Sign Language Understanding (SLU) in computer vision encompasses several research directions: sign language recognition (SLR), sign language translation (SLT), sign spotting, and sign language retrieval.  SLR aims to transcribe a sign video into its constituent glosses (words or phrases representing signs), categorized into isolated SLR (ISLR) for single signs and continuous SLR (CSLR) for sequences of signs. SLT goes further, translating sign languages into spoken languages.  Sign spotting focuses on locating signs within a video, while sign language retrieval involves retrieving relevant sign videos based on textual queries.'}


In [149]:
docs[0].id

'ComputerVision.pdf.1'

In [73]:
response['output_text']

'Sign language understanding in computer vision encompasses several research areas, including sign language recognition (SLR), sign language translation (SLT), sign spotting, and sign language retrieval.  SLR aims to transcribe sign videos into their constituent glosses (words or phrases representing signs), categorized as isolated SLR (ISLR) for single signs and continuous SLR (CSLR) for sequences of signs.  SLT goes further, translating sign languages into spoken languages.  Sign spotting focuses on locating signs within videos, while sign language retrieval involves retrieving relevant sign videos based on textual queries.'

In [139]:
import streamlit as st